Notebook purpose

- Sense-check and explore calculated salaries

In [1]:
import os
import sys
import numpy as np
import pandas as pd
sys.path.append('/Users/fgu/dev/projects/entropy')
import entropy.helpers.aws as aws
import entropy.data.cleaners as cl
import entropy.data.creators as cr

pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 120)
pd.set_option('max_colwidth', None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [4]:
SAMPLE = '777'
fp = f'~/tmp/entropy_{SAMPLE}.parquet'

df = aws.read_parquet(fp)
df.head()

,id,date,user_id,amount,desc,merchant,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,ym,balance,income
0,688261,2012-01-03,777,400.00,<mdbremoved> - s/o,NaN,transfers,False,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,2014-07-18,2017-11-13,True,364.220001,non merchant mbl,transfers,other account,other account,201201,2179.515625,24319.220881
1,688264,2012-01-03,777,10.27,"9572 30dec11 , mcdonalds , restaurant , winwick road gb - pos",mcdonalds,dining and drinking,False,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,2014-07-18,2015-03-19,True,364.220001,mcdonalds,dining and drinking,NaN,dining and drinking,201201,2179.515625,24319.220881
2,688263,2012-01-03,777,6.68,"9572 31dec11 , tesco stores 3345 , warrington gb - pos",tesco,"food, groceries, household",False,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,2014-07-18,2017-08-15,True,364.220001,tesco supermarket,"food, groceries, household",NaN,supermarket,201201,2179.515625,24319.220881
3,688265,2012-01-03,777,12.00,"9572 31dec11 , tesco stores 3345 , warrington gb - pos",tesco,"food, groceries, household",False,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,2014-07-18,2017-08-15,True,364.220001,tesco supermarket,"food, groceries, household",NaN,supermarket,201201,2179.515625,24319.220881
4,688262,2012-01-03,777,3.03,aviva pa - d/d,aviva,health insurance,False,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,2014-07-18,2017-08-15,True,364.220001,aviva,health insurance,life insurance,life insurance,201201,2179.515625,24319.220881


In [19]:
df[df.tag.str.contains('income', na=False)].tag.value_counts().index

CategoricalIndex(['earnings_income', 'benefits_income', 'other_income',
                  'pensions_income', 'interest income', 'personal loan',
                  'payday loan funds', 'payment protection insurance',
                  'paypal account', 'penalty charges',
                  ...
                  'gas', 'flowers', 'food, groceries, household', 'fuel',
                  'furniture', 'furniture, furnishing, gardens', 'gambling',
                  'games and gaming', 'garden', 'zoo/theme park'],
                 categories=['accessories', 'administration - other', 'alcohol', 'appearance', 'appliances or electrical', 'art', 'art supplies', 'bank charges', ...], ordered=False, dtype='category', length=183)

In [87]:
income = ['earnings_income', 'benefits_income', 'other_income', 'pensions_income']
pattern = '|'.join(income)

yearly_salary_payments = (df.loc[df.tag.str.match(pattern)]
                          .set_index('date')
                          .groupby('user_id')
                          .resample('Y'))

yearly_total = yearly_salary_payments.amount.sum()
observed_months = yearly_salary_payments.ym.nunique()

yearly_salary = yearly_total / observed_months * 12

yearly_salary = yearly_salary.rename('salary').reset_index()
yearly_salary['y'] = yearly_salary.date.dt.year
yearly_salary

,user_id,date,salary,y
0,777,2012-12-31,-24319.220881,2012
1,777,2013-12-31,-26577.429688,2013
2,777,2014-12-31,-26179.169922,2014
3,777,2015-12-31,-27538.970703,2015
4,777,2016-12-31,-24164.291016,2016
...,...,...,...,...
147,573777,2020-12-31,-18924.960938,2020
148,578777,2017-12-31,-11636.484375,2017
149,578777,2018-12-31,-7013.640137,2018
150,578777,2019-12-31,-22296.589844,2019


In [90]:
df.tag.isin(['transfers'])

0          True
1         False
2         False
3         False
4         False
          ...  
123759    False
123760     True
123761    False
123762    False
123763    False
Name: tag, Length: 123764, dtype: bool

In [94]:
df.drop(columns='id')

,date,user_id,amount,desc,merchant,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,ym,balance
0,2012-01-03,777,400.000000,<mdbremoved> - s/o,NaN,transfers,False,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,2014-07-18,2017-11-13,True,364.220001,non merchant mbl,transfers,other account,other account,201201,2179.515625
1,2012-01-03,777,10.270000,"9572 30dec11 , mcdonalds , restaurant , winwick road gb - pos",mcdonalds,dining and drinking,False,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,2014-07-18,2015-03-19,True,364.220001,mcdonalds,dining and drinking,NaN,dining and drinking,201201,2179.515625
2,2012-01-03,777,6.680000,"9572 31dec11 , tesco stores 3345 , warrington gb - pos",tesco,"food, groceries, household",False,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,2014-07-18,2017-08-15,True,364.220001,tesco supermarket,"food, groceries, household",NaN,supermarket,201201,2179.515625
3,2012-01-03,777,12.000000,"9572 31dec11 , tesco stores 3345 , warrington gb - pos",tesco,"food, groceries, household",False,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,2014-07-18,2017-08-15,True,364.220001,tesco supermarket,"food, groceries, household",NaN,supermarket,201201,2179.515625
4,2012-01-03,777,3.030000,aviva pa - d/d,aviva,health insurance,False,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,2014-07-18,2017-08-15,True,364.220001,aviva,health insurance,life insurance,life insurance,201201,2179.515625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123759,2020-07-30,578777,5.000000,<mdbremoved> sto,NaN,nan,True,bl8 2,2020-03-29,30k to 40k,1988.0,2020-03-30,1649664,2020-08-16 20:04:00,barclays,current,2020-07-31,1900-01-01,True,266.179993,NaN,NaN,NaN,NaN,202007,243.480103
123760,2020-07-30,578777,3.450000,arthur lane on 29 jul clp,NaN,transfers,True,bl8 2,2020-03-29,30k to 40k,1988.0,2020-03-30,1649664,2020-08-16 20:04:00,barclays,current,2020-07-31,1900-01-01,True,266.179993,NaN,transfers,lunch or snacks,lunch or snacks,202007,243.480103
123761,2020-07-31,578777,15.700000,wickes bury on 30 jul clp,wickes,diy,True,bl8 2,2020-03-29,30k to 40k,1988.0,2020-03-30,1649664,2020-08-16 20:04:00,barclays,current,2020-08-01,1900-01-01,True,266.179993,wickes,diy,NaN,diy,202007,266.180054
123762,2020-07-31,578777,7.000000,pets at home ltd on 30 jul clp,pets at home,pet - everyday or food,True,bl8 2,2020-03-29,30k to 40k,1988.0,2020-03-30,1649664,2020-08-16 20:04:00,barclays,current,2020-08-01,1900-01-01,True,266.179993,pets at home,pet - everyday or food,NaN,pet - everyday or food,202007,266.180054
